In [2]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import os
from tqdm.notebook import tqdm

In [3]:
class Position:
    def __init__(self, posX, posY, isFound, agent):
        self.posX = posX
        self.posY = posY
        self.isFound = isFound
        self.agent = agent
    
    def printData(self):
        print (f"x = {self.posX}\ty = {self.posY}\tfound = {self.isFound}\tagent = {self.agent}")

In [4]:
class Frame:
    def __init__(self):
        self.positions = []
    
    def addPos(self, pos):
        self.positions.append(pos)
        
        
    def printData(self):
        for pos in self.positions:
            pos.printData()
        print("\n")

In [5]:
class Video:
    def __init__(self):
        self.frames = []
        
    def addFrame(self, frame):
        self.frames.append(frame)
        
    def printData(self):
        for frame in self.frames:
            frame.printData()

In [6]:
class AgentTracker:
    def __init__(self, vid):
        self.numAgents = len(vid.frames[0].positions)
        self.agents = []
        for x in range(self.numAgents):
            self.agents.append(Agent(vid, x))
    
    def AgentTrackerToVid(self):
        vid = Video()
        numFrames = len(self.agents[0].positions)
        for frameNumber in range(numFrames):
            frame = Frame()
            for agent in self.agents:
                frame.addPos(agent.positions[frameNumber])
            vid.addFrame(frame)
        return vid

In [7]:
class Agent:              
    def __init__(self, vid, agentID):
        self.positions = []
        self.agentName = vid.frames[0].positions[agentID].agent
        for frame in vid.frames:
            self.positions.append(frame.positions[agentID])
        self.fillInGaps()
    
    def fillInGaps(self):
        for x in range(len(self.positions)):
            pos = self.positions
            if (pos[x].isFound):
                prevFrameNum = x - 1
                while (prevFrameNum >= 0 and not pos[prevFrameNum].isFound):
                    prevFrameNum -= 1
                if (prevFrameNum == -1):
                    for y in range(0, x):
                        pos[y] = pos[x]
                else:
                    gapFrameCount = x - prevFrameNum
                    incrementX = (pos[x].posX - pos[prevFrameNum].posX) / gapFrameCount
                    incrementY = (pos[x].posY - pos[prevFrameNum].posY) / gapFrameCount
                    for increment in range(1, gapFrameCount):
                        pos[increment + prevFrameNum].posX = pos[prevFrameNum].posX + (increment * incrementX)
                        pos[increment + prevFrameNum].posY = pos[prevFrameNum].posY + (increment * incrementY)
                        pos[increment + prevFrameNum].isFound = True

In [8]:
def getAgentFiles():   
    folders = next(os.walk('./agent_icons'))[1] #get all the folders
    folders.pop() #remove ipynb and old folder
    folders.pop(0)
    
    allFileNames = []
    for folder in folders:
        agentIconsFileList = os.listdir(f"./agent_icons/{folder}")
        for x in range(len(agentIconsFileList)):
            agentIconsFileList[x] = f"agent_icons/{folder}/" + agentIconsFileList[x] #list for each agent
        allFileNames.append(agentIconsFileList)
    return allFileNames

In [36]:
def trackAgent(img_gray, img_rgb, agent_file, threshold):
    minimap = cv2.imread('./maps/Ascent_Minimap.png', cv2.IMREAD_COLOR)
    agent_name = agent_file.split("_")[1].split("/")[2]
    subImage = cv2.imread(agent_file, 0)
    w, h = subImage.shape[::-1]
    res = cv2.matchTemplate(img_gray, subImage,cv2.TM_CCOEFF_NORMED)
    loc = np.where( res >= threshold)
    pos = Position(0, 0, False, agent_name.split(".")[0])
    for pt in zip(*loc[::-1]):
        SCALE_FACTOR = 3.7275
        SHIFT_FACTOR = -103
        newX = (int ((pt[0] + 0.5 * w) * SCALE_FACTOR)) + SHIFT_FACTOR
        newY = (int ((pt[1] + 0.5 * h) * SCALE_FACTOR)) + SHIFT_FACTOR
        if (not (np.all(minimap[newX, newY] == [0, 0, 0]))):
            pos = Position(newX, newY, True, agent_name)
        
    return (pos)

In [10]:
def trackAllAgents(img_gray, img_rgb, threshold):
    frame = Frame()
    for agent in getAgentFiles():
        agent_name = agent[0].split("_")[1].split("/")[2]
        found = False
        for icon in agent:
            pos = trackAgent(img_gray, img_rgb, icon, threshold)
            if (pos.isFound):
                found = True
                break;
        if (not found):
            pos = Position(0, 0, False, agent_name)
        frame.addPos(pos)

    return (frame)

In [32]:
def visualizeData(vid):
    count = 0
    for frame in vid.frames:
        count += 1
        if (count > 150):
            break
        minimap = cv2.imread('./maps/Ascent_Minimap.png', cv2.IMREAD_COLOR)
        for pos in frame.positions:
            if (pos.isFound):
                minimap = cv2.circle(minimap, (int(pos.posX), int(pos.posY)) , radius=5, color=(0, 0, 255), thickness=-1)
        minimap = cv2.resize(minimap, (0,0), fx=0.5, fy=0.5)
        cv2.imshow('map', minimap)
        cv2.waitKey(33)
    cv2.destroyAllWindows()

In [ ]:
#all icons, video
cap = cv2.VideoCapture("game.mp4")
vid = Video()
START_FRAME = 150
END_FRAME = 300

cap.set(3, 1280)
cap.set(4, 720)
cap.set(1, START_FRAME)

currentFrame = 0
for x in tqdm(range(0, END_FRAME)):
    ret, frame = cap.read()
    frame = frame[0:300, 0:300]
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    vid.addFrame(trackAllAgents(img_gray, frame, 0.95))
# visualizeData(vid)
# cv2.destroyAllWindows()

In [13]:
# visualizeData(vid)
vid.printData()

x = 0	y = 0	found = False	agent = breach
x = 348	y = 612	found = True	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 586	y = 769	found = True	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 690	y = 806	found = True	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 0	y = 0	found = False	agent = breach
x = 348	y = 612	found = True	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 586	y = 769	found = True	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 690	y = 806	found = True	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 0	y = 0	found = False	agent = breach
x = 348	y = 612	found = True	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 586	y = 769	found = True	agent = omen
x = 0	y = 0	found = F

x = 0	y = 0	found = False	agent = jett
x = 616	y = 769	found = True	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 690	y = 806	found = True	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 0	y = 0	found = False	agent = breach
x = 0	y = 0	found = False	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 616	y = 769	found = True	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 690	y = 806	found = True	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 0	y = 0	found = False	agent = breach
x = 0	y = 0	found = False	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 616	y = 769	found = True	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 690	y = 806	found = True	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True

x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 0	y = 0	found = False	agent = breach
x = 0	y = 0	found = False	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 627	y = 157	found = True	agent = jett
x = 0	y = 0	found = False	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 690	y = 806	found = True	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 346	y = 601	found = True	agent = breach
x = 0	y = 0	found = False	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 653	y = 772	found = True	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 0	y = 0	found = False	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 0	y = 0	found = False	agent = breach
x = 0	y = 0	found = False	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 0	y = 0	found = False

x = 0	y = 0	found = False	agent = jett
x = 664	y = 750	found = True	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 0	y = 0	found = False	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 0	y = 0	found = False	agent = breach
x = 0	y = 0	found = False	agent = breach2
x = 362	y = 750	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 664	y = 750	found = True	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 0	y = 0	found = False	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 0	y = 0	found = False	agent = breach
x = 0	y = 0	found = False	agent = breach2
x = 362	y = 754	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 661	y = 746	found = True	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 0	y = 0	found = False	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = 

x = 0	y = 0	found = False	agent = breach
x = 0	y = 0	found = False	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 664	y = 717	found = True	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 0	y = 0	found = False	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 0	y = 0	found = False	agent = breach
x = 0	y = 0	found = False	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 0	y = 0	found = False	agent = omen
x = 0	y = 0	found = False	agent = phoneix
x = 0	y = 0	found = False	agent = redsova
x = 355	y = 359	found = True	agent = sage
x = 232	y = 348	found = True	agent = sova


x = 0	y = 0	found = False	agent = breach
x = 355	y = 538	found = True	agent = breach2
x = 847	y = 426	found = True	agent = cypher
x = 0	y = 0	found = False	agent = jett
x = 0	y = 0	found = False	agent = omen
x = 0	y = 0	found = False	agent = phone

In [35]:
adjVid = AgentTracker(vid)
adjVid = adjVid.AgentTrackerToVid()
visualizeData(adjVid)
# adjVid.printData()